In [0]:
%pip install -r requirements.txt
%restart_python

In [0]:
%restart_python

In [0]:
from src.ingest_data import load_and_prepare_data
from src.forecast import train_and_forecast
import json
import pandas as pd
from pathlib import Path
import mlflow
import random
import numpy as np


# Set seeds
random.seed(42)
np.random.seed(42)

In [0]:
CONFIG_PATH = Path("configs/")

def run_all_models_forecast():
    all_forecasts = []
    for config_file in CONFIG_PATH.glob("*.json"):
        with open(config_file) as f:
            params = json.load(f)

        model_name = params["model_name"]
        target = params['target']
        horizon = params['horizon']

        print(f"Running pipeline for: {model_name}")

        # Load and prepare data
        df = load_and_prepare_data(
            path="data/processed/combined_total_household_data_interpolate.csv",
            target=target,
            params=params
        )

        # Train the model and forecast
        forecast_df, model = train_and_forecast(df, h=horizon, param_dict=params, target = target)
        
        # Label the forecast by the intended horizon (not the underlying `fh`)
        forecast_df["horizon"] = horizon
        all_forecasts.append(forecast_df)

    # Combine all forecasts
    final_forecast_df = (
    pd.concat(all_forecasts)
    .sort_index()
    .reset_index()
    .rename(columns={"index": "date"}))

    # Save the forecast outputs
    path_dir = Path("outputs/forecasts")
    path_dir.mkdir(parents=True, exist_ok=True)
    filepath = path_dir / "final_forecast_df.csv"
    final_forecast_df.to_csv(filepath, index = False)

    return final_forecast_df


In [0]:
final_forecast_df = run_all_models_forecast()
final_forecast_df